In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
def get_raw_html(url: str) -> any:
    response = requests.get(url)
    response.encoding='utf-8'
    return response

In [3]:
city = "ufa"
car_name = "vaz"
url_basic = "https://auto.ru/{}/cars/{}/all/?query=%D0%B2%D0%B0%D0%B7&from=searchline".format(city, car_name)
response = get_raw_html(url_basic)

In [4]:
bs_obj = BeautifulSoup(response.text, 'html.parser')

In [5]:
page_numbers = bs_obj.select('.ListingPagination__page')
count_pages = max([int(nth_page.get_text()) for nth_page in page_numbers if nth_page.get_text().isdigit()])

In [6]:
max_n_pages = 100
max_n_pages = min(max_n_pages, count_pages)

In [7]:
max_n_pages

52

In [8]:
def clean_str(str_expr: str) -> str:
    return re.sub(r'\u2009|\xa0','',str_expr)

def add_record(df: dict, item) -> None:
    name = item.select('.ListingItemTitle__link')[0].get_text()
    df['Name'].append(name)
    url = item.select('.ListingItemTitle__link')[0].get('href')
    df['URL'].append(url)
    price = clean_str(item.select('.ListingItemPrice__content')[0].get_text())
    price = price.replace('₽','')
    df['Price, rub'].append(price)
    year = item.select('.ListingItem__year')[0].get_text()
    df['Year'].append(year)
    kmAge = clean_str(item.select('.ListingItem__kmAge')[0].get_text())
    kmAge = kmAge.replace('км','')
    df['KmAge'].append(kmAge)
    chrts = chrts = item.select('.ListingItemTechSummaryDesktop__column')[0].select('.ListingItemTechSummaryDesktop__cell')
    engine = clean_str(chrts[0].get_text())
    gearBox = clean_str(chrts[1].get_text())
    carType= clean_str(chrts[2].get_text())
    df['Engine'].append(engine)
    df['GearBox'].append(gearBox)
    df['CarType'].append(carType)
    place = clean_str(item.select('.MetroListPlace__regionName')[0].get_text())
    df['Place'].append(place)
    

In [9]:
data = {'Name' : [], 'URL': [], 'Price, rub': [], 'Year': [], 'KmAge': [], 'Engine': [], 'GearBox': [], 'CarType': [],
       'Place': []}

for page_number in range(1, max_n_pages + 1):
    url_page_nth = ("https://auto.ru/{}/cars/{}/all/?query=%D0%BB%D0%B0%D0%B4%D0%B0&from=searchline&page={}".
        format(city, car_name, page_number))
    response = get_raw_html(url_page_nth)
    bs_obj = BeautifulSoup(response.text, 'html.parser')
    items = bs_obj.select('.ListingItem__description')
    for item in items:
        add_record(data, item)

In [11]:
data_df = pd.DataFrame.from_dict(data)
data_df.to_csv('parsed_data.csv',encoding='cp1251',index=False)